# Analyse de règles d'association dans un magasin agroalimentaire
---

## Table des matières

1. [Contexte et objectifs](#1-contexte-et-objectifs)
2. [Formulation du problème](#2-formulation-du-problème)
3. [Chargement et exploration des données](#3-chargement-et-exploration-des-données)
4. [Application de l'algorithme Apriori](#4-application-de-lalgorithme-apriori)
5. [Application de l'algorithme FP-Growth](#5-application-de-lalgorithme-fp-growth)
6. [Comparaison des algorithmes](#6-comparaison-des-algorithmes)
7. [Interprétation et recommandations](#7-interprétation-et-recommandations)
8. [Conclusion](#8-conclusion)

## **1. Contexte et objectifs**

### Contexte

Un magasin agroalimentaire souhaite optimiser l'attribution de promotions sur ses produits alimentaires. L'objectif est de mieux comprendre les habitudes d'achat des clients, en identifiant les produits qui sont fréquemment achetés ensemble.

### Objectifs pédagogiques

- Découvrir et manipuler les techniques de fouille de règles d'association
- Comprendre les notions de **support**, **confiance** et **lift**
- Comparer deux algorithmes populaires : **Apriori** et **FP-Growth**
- Interpréter les règles obtenues et formuler des recommandations marketing

## **2. Formulation du problème**

### Définition du problème

Le problème :

> **"Trouver des ensembles de produits fréquemment achetés ensemble, et identifier les règles qui permettent de prédire l'achat d'un produit à partir de la présence d'autres produits dans le panier."**

### Représentation des données

Les tickets de caisse du magasin sont représentés sous forme de **transactions**, où chaque transaction correspond à un panier d'achat contenant un ensemble de produits.

**Exemple de transaction :**
```
Transaction 1: {pain, fromage, vin}
Transaction 2: {lait, céréales, jus}
```

### Règles d'association

Une règle d'association a la forme : **A --> B**

- **A** (antécédent) : produit(s) déjà dans le panier
- **B** (conséquent) : produit(s) dont l'achat est prédit

**Exemple :** `{pain, fromage} --> {vin}`

Cette règle signifie : "Si un client achète du pain et du fromage, il est probable qu'il achète aussi du vin."

### Métriques d'évaluation

1. **Support** : Fréquence d'apparition des éléments ensemble
   - High support = combinaison fréquente
   - `Support(A --> B) = P(A ∪ B) = nombre de transactions contenant A et B / nombre total de transactions`
   - Ex.: Si 20 transactions sur 100 contiennent `{pain, beurre}`, le support est de 20% = 0,2.
   
2. **Confiance** : Probabilité que B apparaisse sachant que A est présent
   - Dans quelle mesure cette règle est-elle fiable ? « Lorsque le produit A est acheté, à quelle fréquence le produit B est-il également acheté ? »
   - `Confiance(A --> B) = P(B|A) = Support(A ∪ B) / Support(A)`
   - Ex.: Si 80 % des personnes qui achètent du pain achètent aussi du beurre, le taux de confiance est de 0,8.

3. **Lift** : Force de l'association entre A et B
   - Cette association est-elle significative ou simplement aléatoire ?
   - `Lift(A --> B) = Confiance(A --> B) / Support(B)`
   - Lift > 1 : corrélation positive (association intéressante)
   - Lift = 1 : pas de corrélation
   - Lift < 1 : corrélation négative

## **3. Chargement et exploration des données**

### Import des bibliothèques

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for French display
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
sns.set_style("whitegrid")

In [2]:
# Import my functions
import association_rules as ar

### Chargement des données

In [3]:
# Load and prepare data
filepath = 'transactions_promos.csv'
df_encoded, transactions = ar.load_and_prepare_data(filepath)

print("✓ Données chargées et préparées avec succès")

✓ Données chargées et préparées avec succès


### Exploration des données

In [4]:
# Display basic information about the dataset
ar.display_data_info(df_encoded, transactions)

INFORMATIONS SUR LES DONNÉES
Nombre total de transactions : 40
Nombre total de produits uniques : 44

Premières transactions :
  Transaction 1: bread, butter, milk, eggs
  Transaction 2: bread, butter, jam
  Transaction 3: bread, milk, cereal
  Transaction 4: milk, cereal, banana
  Transaction 5: bread, cheese, tomato




### Fréquence des produits individuels

In [5]:
# Get item frequencies
item_freq = ar.get_item_frequencies(df_encoded)

print("=" * 60)
print("FRÉQUENCE DES PRODUITS INDIVIDUELS")
print("=" * 60)
print(item_freq.head(15).to_string(index=False))
print("=" * 60)

FRÉQUENCE DES PRODUITS INDIVIDUELS
     Produit  Fréquence  Support
       bread          9    0.225
        milk          9    0.225
      butter          6    0.150
    biscuits          5    0.125
       pasta          5    0.125
      cheese          5    0.125
      banana          4    0.100
        rice          4    0.100
      coffee          4    0.100
        eggs          4    0.100
         tea          4    0.100
       sugar          4    0.100
tomato_sauce          4    0.100
      tomato          4    0.100
      yogurt          3    0.075


### Visualisation des produits les plus fréquents

In [6]:
# Visualize top products
def frequent_products(top_products: pd.DataFrame):
    """
    Plot frequent products.
    
    Args:
        top_products: Dataframe with the frequency of each individual product
    """
    fig = px.bar(
        top_products, 
        x='Support', 
        y='Produit',
        orientation='h',
        title='Top 15 Produits les Plus Fréquents',
        text='Support'
    )

    fig.update_traces(
        marker_color='lightsteelblue',
        texttemplate='%{text:.2%}',
    )

    fig.update_layout(
        height=600,
        title_font_size=18,
        title_x=0.5,  ## to the middle
        xaxis_title='Support',
        yaxis_title='Produit',
        yaxis={'categoryorder':'total ascending'},
        template='plotly_white',
    )

    fig.show()

top_products = item_freq.head(15)
frequent_products(top_products)

#### **Observations:** 

**Produits dominants :**
- Le lait et le pain ont un support exceptionnellement élevé (22,5%) comparé à tous les autres produits de la liste, apparaissant dans environ 1 transaction sur 4 (9 transactions sur 40)
- Ce sont des produits essentiels du quotidien

**Produits moins fréquents :**
- Le beurre avec 15% de support
- Le fromage, les pâtes et les biscuits avec 12,50% de support - apparaissant dans environ 1 transaction sur 8 (5 transactions sur 40)

**Insights :**
- Le top 15 des produits ne contient aucun article hautement spécialisé ou de niche
- La plupart des produits sont des ingrédients pour des repas de base :
  - **Pâtisserie :** farine, beurre, sucre, oeufs, lait
  - **Repas froids :** pain, beurre, fromage, tomate, oeufs
  - **Moment goûter :** café, thé, biscuits, yaourt, sucre, banane, lait
  - **Bases culinaires :** lait, beurre, fromage, pâtes, sauce tomate, oeufs, riz

#### Implications / Hypothèses sur les règles d'association

- Le seuil de support devra être fixé en dessous de 20%, car seulement 2 produits ont des scores de support individuels dépassant 20%. Par conséquent, il est peu probable que les combinaisons de produits atteignent un support supérieur aux produits individuels.
- Les règles impliquant le lait et le pain auront probablement des scores de support élevés.
- Il sera intéressant d'explorer les associations au sein des groupements d'ingrédients mentionnés ci-dessus (repas froids, moment goûter, bases culinaires/pâtisserie).

## **4. Application de l'algorithme Apriori**

### Principe de l'algorithme Apriori

L'algorithme **Apriori** utilise une approche itérative pour découvrir les ensembles fréquents :
1. Trouver tous les 1-itemsets fréquents (produits individuels)
2. Générer les candidats 2-itemsets à partir des 1-itemsets
3. Calculer le support des candidats et filtrer
4. Répéter pour les 3-itemsets, 4-itemsets, etc.

**Propriété clé :** Si un itemset est infrequent, tous ses sur-ensembles sont aussi infrequents.

### Définition des seuils

La meilleure pratique pour déterminer les seuils de support et de confiance consiste à consulter des experts du domaine et à appliquer des valeurs qui ont du sens dans le contexte métier spécifique. Cependant, en l'absence d'expertise sectorielle, j'adopte une approche systématique pour identifier les seuils optimaux : la **recherche par grille (grid search)**.

#### Objectifs de l'optimisation

Je cherche à trouver un équilibre entre trois objectifs:

1. **Suffisamment de règles** : Un nombre minimal de règles (≥ 10) est nécessaire pour obtenir des insights exploitables et identifier des patterns variés dans les données.

2. **Pas trop de règles** : Un nombre excessif de règles (> 50) rend l'analyse difficile, et crée de la redondance.

3. **Règles de haute qualité** : Les règles doivent représenter des associations réelles et significatives, pas simplement des coincidences statistiques.

#### Indicateurs de qualité

Comme mentionné précédemment dans la formulation du problème, le **lift** est l'indicateur clé pour évaluer la qualité d'une règle d'association :

- **Lift = 1** : Les produits apparaissent ensemble par pur hasard (absence d'association)
- **Lift > 1** : Corrélation positive - les produits sont achetés ensemble plus fréquemment que si les achats étaient indépendants
- **Lift >> 1** : Association forte

**Interprétation pratique :** Un lift de 1.5 signifie que les clients achètent ces produits ensemble 50% plus souvent que ce que le hasard prédirait. Un lift de 2.0 indique qu'ils les achètent ensemble deux fois plus souvent.

En plus du lift, j'analyse :
- Le **nombre total de règles générées** pour évaluer si mes seuils sont appropriés
- Le **lift moyen** de l'ensemble des règles comme indicateur de qualité globale

#### Le compromis (trade-off)

Il existe un compromis dans le choix des seuils :

- **Support élevé + Confiance élevée** --> Peu de règles, mais très robustes et fiables
- **Support faible + Confiance faible** --> Beaucoup de règles, mais risque d'inclure des associations faibles ou non pertinentes

L'objectif est de trouver l'équilibre : des seuils qui génèrent un nombre gérable de règles tout en maximisant leur pertinence métier (lift élevé).

#### Méthodologie de recherche

Je teste différentes combinaisons de seuils dans les plages suivantes :
- **Support** : de 5% à 23% (adaptation à la taille du dataset)
- **Confiance** : de 50% à 80%

Pour chaque combinaison, j'évalue :
1. Le nombre de règles générées
2. Le lift moyen de ces règles
3. La présence de règles avec un lift significativement supérieur à 1

Les seuils retenus seront ceux qui offrent le meilleur équilibre entre quantité et qualité de règles.

In [7]:
# Test different threshold combinations
# support_values = np.round(np.linspace(0.05, 0.23, 19), 2).tolist()
support_values = np.round(np.linspace(0.05, 0.11, 7), 2).tolist()  ## step 0.01
confidence_values = np.round(np.linspace(0.50, 0.80, 7), 2).tolist()  ## step: 0.05

### Exécution d'Apriori

In [8]:
results = []

for sup in support_values:
    for conf in confidence_values:
        itemsets, rules, time = ar.run_apriori(df_encoded, min_support=sup, min_confidence=conf, display=False)
        results.append({
            'support': sup,
            'confidence': conf,
            'num_itemsets': len(itemsets),
            'num_rules': len(rules),
            'avg_lift': rules['lift'].mean() if len(rules) > 0 else 0
        })

In [9]:
# Display results
results_df = pd.DataFrame(results)
print(results_df)

# Find the right parameters - good number of rules with high lift
results_df[results_df['num_rules'].between(5, 50)].sort_values('avg_lift', ascending=False)

    support  confidence  num_itemsets  num_rules  avg_lift
0      0.05        0.50            48         34  5.331155
1      0.05        0.55            48         18  6.008230
2      0.05        0.60            48         18  6.008230
3      0.05        0.65            48         17  6.204793
4      0.05        0.70            48          8  6.333333
5      0.05        0.75            48          7  6.761905
6      0.05        0.80            48          7  6.761905
7      0.06        0.50            25          6  4.901235
8      0.06        0.55            25          6  4.901235
9      0.06        0.60            25          6  4.901235
10     0.06        0.65            25          5  5.348148
11     0.06        0.70            25          4  5.944444
12     0.06        0.75            25          4  5.944444
13     0.06        0.80            25          3  6.814815
14     0.07        0.50            25          6  4.901235
15     0.07        0.55            25          6  4.9012

,support,confidence,num_itemsets,num_rules,avg_lift
6,0.05,0.80,48,7,6.761905
5,0.05,0.75,48,7,6.761905
4,0.05,0.70,48,8,6.333333
3,0.05,0.65,48,17,6.204793
1,0.05,0.55,48,18,6.008230
2,0.05,0.60,48,18,6.008230
10,0.06,0.65,25,5,5.348148
17,0.07,0.65,25,5,5.348148
0,0.05,0.50,48,34,5.331155
8,0.06,0.55,25,6,4.901235




Sur la base des résultats de la recherche par grille, j'ai conclu que **support 0,05 (5%)** avec **confiance 0,65** offre le meilleur équilibre :

- L'algorithme ne peut trouver aucune règle lorsque le support est égal ou supérieur à 0,11 (11%), ce qui signifie qu'il n'existe aucune combinaison de produits apparaissant plus de 4 fois ensemble dans le dataset. Ce seuil est bien trop restrictif pour un dataset aussi petit.

- Étant donné la taille du dataset (40 transactions), un seuil de support de 5% signifie que les combinaisons de produits doivent apparaître dans au moins 2 transactions, ce qui est raisonnable.

- 17 règles sont générées avec une confiance de 65%, ce qui est suffisant pour l'analyse tout en maintenant une fiabilité élevée. Des seuils de confiance plus bas (0,60) produisent 18 règles, mais le gain marginal est minimal, et une confiance plus élevée garantit des associations plus fiables.

- Le lift moyen est d'environ 6,2, ce qui indique des associations très fortes.

In [10]:
MIN_SUPPORT = 0.05     # 5% - at least 2 transactions out of 40
MIN_CONFIDENCE = 0.65  # 65% - will have 17 rules

In [11]:
# Run Apriori algorithm
apriori_itemsets, apriori_rules, apriori_time = ar.run_apriori(
    df_encoded, 
    min_support=MIN_SUPPORT, 
    min_confidence=MIN_CONFIDENCE
)

EXÉCUTION DE L'ALGORITHME APRIORI
Support minimal : 5.0%
Confiance minimale : 65.0%

Traitement en cours...
✓ Terminé en 0.0053 secondes
✓ 48 ensembles fréquents trouvés
✓ 17 règles d'association générées



### Ensembles fréquents trouvés

In [12]:
# Display frequent itemsets
ar.display_frequent_itemsets(apriori_itemsets, top_n=15)


TOP 15 ENSEMBLES FRÉQUENTS (par support)
 1. {bread}                    | Support: 0.225 | Taille: 1
 2. {milk}                     | Support: 0.225 | Taille: 1
 3. {butter}                   | Support: 0.150 | Taille: 1
 4. {cheese}                   | Support: 0.125 | Taille: 1
 5. {pasta}                    | Support: 0.125 | Taille: 1
 6. {biscuits}                 | Support: 0.125 | Taille: 1
 7. {coffee}                   | Support: 0.100 | Taille: 1
 8. {banana}                   | Support: 0.100 | Taille: 1
 9. {eggs}                     | Support: 0.100 | Taille: 1
10. {pasta, tomato_sauce}      | Support: 0.100 | Taille: 2
11. {bread, butter}            | Support: 0.100 | Taille: 2
12. {tomato_sauce}             | Support: 0.100 | Taille: 1
13. {tea}                      | Support: 0.100 | Taille: 1
14. {tomato}                   | Support: 0.100 | Taille: 1
15. {sugar}                    | Support: 0.100 | Taille: 1



In [13]:
ar.display_frequent_list_itemsets(apriori_itemsets, top_n=15)

TOP 15 COMBINAISONS DE PRODUITS (taille > 1)
Nombre total d'itemsets: 48
Nombre d'itemsets de produits combinés: 24

 1. {bread, butter}            | Support: 0.100 | Taille: 2
 2. {pasta, tomato_sauce}      | Support: 0.100 | Taille: 2
 3. {bread, cheese}            | Support: 0.075 | Taille: 2
 4. {banana, milk}             | Support: 0.075 | Taille: 2
 5. {cereal, milk}             | Support: 0.075 | Taille: 2
 6. {biscuits, coffee}         | Support: 0.050 | Taille: 2
 7. {biscuits, tea}            | Support: 0.050 | Taille: 2
 8. {banana, cereal}           | Support: 0.050 | Taille: 2
 9. {bread, jam}               | Support: 0.050 | Taille: 2
10. {bread, eggs}              | Support: 0.050 | Taille: 2
11. {butter, eggs}             | Support: 0.050 | Taille: 2
12. {bread, milk}              | Support: 0.050 | Taille: 2
13. {butter, jam}              | Support: 0.050 | Taille: 2
14. {cheese, pasta}            | Support: 0.050 | Taille: 2
15. {cheese, tomato}           | Support: 0

### Règles d'association générées

In [14]:
# Display association rules sorted by lift
ar.display_rules(apriori_rules, top_n=15, sort_by='lift')

,Règle,Support,Confiance,Lift
1,{flour} --> {eggs},0.050,1.000000,10.000000
2,"{cereal} --> {banana, milk}",0.050,0.666667,8.888889
3,"{banana, milk} --> {cereal}",0.050,0.666667,8.888889
4,{pasta} --> {tomato_sauce},0.100,0.800000,8.000000
5,"{bread, tomato} --> {cheese}",0.050,1.000000,8.000000
6,{tomato_sauce} --> {pasta},0.100,1.000000,8.000000
7,{cereal} --> {banana},0.050,0.666667,6.666667
8,{honey} --> {tea},0.050,0.666667,6.666667
9,"{bread, cheese} --> {tomato}",0.050,0.666667,6.666667
10,"{cereal, milk} --> {banana}",0.050,0.666667,6.666667


### Analyse de la qualité des règles Apriori

In [15]:
# Analyse rule quality
apriori_quality = ar.analyse_rule_quality(apriori_rules)
ar.display_quality_analysis(apriori_quality)

ANALYSE DE LA QUALITÉ DES RÈGLES
Nombre total de règles : 17

Corrélation positive (lift > 1) : 17 règles
Corrélation négative (lift < 1) : 0 règles
Pas de corrélation (lift = 1)   : 0 règles

Moyennes :
  Lift moyen       : 6.205
  Confiance moyenne: 0.797
  Support moyen    : 0.062



### Interprétation des principales règles Apriori

L'algorithme a généré 17 règles avec les seuils suivants :
- Support minimal : 5%
- Confiance minimale : 65%


#### Règles avec Lift > 8

**{flour} --> {eggs}** (support = 5%, confiance = 100%, lift = 10,0)
- Indique que tous les clients qui achètent de la farine achètent également des oeufs
- Un lift de 10 signifie que cette paire apparaît 10 fois plus souvent que si les achats étaient indépendants
- Comme prévu, ces produits sont des incontournables de la pâtisserie, il est donc cohérent qu'ils soient achetés ensemble

**{cereal} <--> {banana, milk}** (support = 5%, confiance = 67%, lift = 8,9)
- Lorsqu'un client achète des céréales, il y a 67% de chances qu'il achète aussi des bananes et du lait, OU
- Lorsqu'un client achète des bananes et du lait, il y a 67% de chances qu'il achète aussi des céréales
- Cela démontre une forte association bidirectionnelle entre les deux ensembles
- Bien que les céréales ne figurent pas parmi les produits individuels les plus fréquents, ce trio représente un panier petit-déjeuner complet

**{pasta} <--> {tomato_sauce}** (support = 10%, confiance = 80-100%, lift = 8,0)
- Forte association bidirectionnelle entre les pâtes et la sauce tomate
- Tous les clients qui achètent de la sauce tomate achètent également des pâtes (la confiance de {tomato_sauce} --> {pasta} est de 100%)

**{bread, tomato} --> {cheese}** (support = 5%, confiance = 100%, lift = 8,0)
- Lorsqu'un client achète du pain et des tomates, il achète toujours du fromage également
- Suggère fortement que ces articles sont consommés ensemble comme un repas froid


#### Observations générales

1. **Thèmes récurrents** : Comme anticipé, les règles se regroupent autour d'occasions de repas spécifiques - pâtisserie, petit-déjeuner/repas froids, cuisine, moment goûter, etc.

2. **Confiance élevée** : La majorité des règles ont une confiance supérieure à 66%, ce qui les rend fiables pour générer des recommandations

3. **Associations logiques** : Un plan de repas cohérent peut être déduit des règles

4. **Bidirectionnalité** : Indique des associations particulièrement fortes entre les produits

**Conclusion** : Ces règles sont fiables et constituent une base solide pour des recommandations marketing éclairées.

## **5. Application de l'algorithme FP-Growth**

### Principe de l'algorithme FP-Growth

L'algorithme **FP-Growth** (Frequent Pattern Growth) utilise une structure de données compacte appelée **FP-Tree** :

1. Construction du FP-Tree à partir des transactions
2. Extraction récursive des patterns fréquents
3. Pas de génération de candidats (contrairement à Apriori)

### Exécution de FP-Growth

In [16]:
# Run FP-Growth algorithm with same parameters
fpgrowth_itemsets, fpgrowth_rules, fpgrowth_time = ar.run_fpgrowth(
    df_encoded, 
    min_support=MIN_SUPPORT, 
    min_confidence=MIN_CONFIDENCE
)

EXÉCUTION DE L'ALGORITHME FP-GROWTH
Support minimal : 5.0%
Confiance minimale : 65.0%

Traitement en cours...
✓ Terminé en 0.0086 secondes
✓ 48 ensembles fréquents trouvés
✓ 17 règles d'association générées



### Ensembles fréquents trouvés

In [17]:
# Display frequent itemsets
ar.display_frequent_itemsets(fpgrowth_itemsets, top_n=15)


TOP 15 ENSEMBLES FRÉQUENTS (par support)
 1. {bread}                    | Support: 0.225 | Taille: 1
 2. {milk}                     | Support: 0.225 | Taille: 1
 3. {butter}                   | Support: 0.150 | Taille: 1
 4. {cheese}                   | Support: 0.125 | Taille: 1
 5. {biscuits}                 | Support: 0.125 | Taille: 1
 6. {pasta}                    | Support: 0.125 | Taille: 1
 7. {eggs}                     | Support: 0.100 | Taille: 1
 8. {banana}                   | Support: 0.100 | Taille: 1
 9. {coffee}                   | Support: 0.100 | Taille: 1
10. {rice}                     | Support: 0.100 | Taille: 1
11. {tomato_sauce}             | Support: 0.100 | Taille: 1
12. {tomato}                   | Support: 0.100 | Taille: 1
13. {sugar}                    | Support: 0.100 | Taille: 1
14. {pasta, tomato_sauce}      | Support: 0.100 | Taille: 2
15. {bread, butter}            | Support: 0.100 | Taille: 2



In [18]:
ar.display_frequent_list_itemsets(apriori_itemsets, top_n=15)

TOP 15 COMBINAISONS DE PRODUITS (taille > 1)
Nombre total d'itemsets: 48
Nombre d'itemsets de produits combinés: 24

 1. {bread, butter}            | Support: 0.100 | Taille: 2
 2. {pasta, tomato_sauce}      | Support: 0.100 | Taille: 2
 3. {bread, cheese}            | Support: 0.075 | Taille: 2
 4. {banana, milk}             | Support: 0.075 | Taille: 2
 5. {cereal, milk}             | Support: 0.075 | Taille: 2
 6. {biscuits, coffee}         | Support: 0.050 | Taille: 2
 7. {biscuits, tea}            | Support: 0.050 | Taille: 2
 8. {banana, cereal}           | Support: 0.050 | Taille: 2
 9. {bread, jam}               | Support: 0.050 | Taille: 2
10. {bread, eggs}              | Support: 0.050 | Taille: 2
11. {butter, eggs}             | Support: 0.050 | Taille: 2
12. {bread, milk}              | Support: 0.050 | Taille: 2
13. {butter, jam}              | Support: 0.050 | Taille: 2
14. {cheese, pasta}            | Support: 0.050 | Taille: 2
15. {cheese, tomato}           | Support: 0

### Règles d'association générées

In [19]:
# Display association rules sorted by lift
ar.display_rules(fpgrowth_rules, top_n=15, sort_by='lift')

,Règle,Support,Confiance,Lift
1,{flour} --> {eggs},0.050,1.000000,10.000000
2,"{banana, milk} --> {cereal}",0.050,0.666667,8.888889
3,"{cereal} --> {banana, milk}",0.050,0.666667,8.888889
4,{pasta} --> {tomato_sauce},0.100,0.800000,8.000000
5,{tomato_sauce} --> {pasta},0.100,1.000000,8.000000
6,"{bread, tomato} --> {cheese}",0.050,1.000000,8.000000
7,"{bread, cheese} --> {tomato}",0.050,0.666667,6.666667
8,{cereal} --> {banana},0.050,0.666667,6.666667
9,"{cereal, milk} --> {banana}",0.050,0.666667,6.666667
10,{honey} --> {tea},0.050,0.666667,6.666667


### Analyse de la qualité des règles FP-Growth

In [20]:
# Analyse rule quality
fpgrowth_quality = ar.analyse_rule_quality(fpgrowth_rules)
ar.display_quality_analysis(fpgrowth_quality)

ANALYSE DE LA QUALITÉ DES RÈGLES
Nombre total de règles : 17

Corrélation positive (lift > 1) : 17 règles
Corrélation négative (lift < 1) : 0 règles
Pas de corrélation (lift = 1)   : 0 règles

Moyennes :
  Lift moyen       : 6.205
  Confiance moyenne: 0.797
  Support moyen    : 0.062



## **6. Comparaison des algorithmes**

### Tableau comparatif

In [21]:
# Compare algorithms
comparison = ar.compare_algorithms(
    (apriori_itemsets, apriori_rules, apriori_time),
    (fpgrowth_itemsets, fpgrowth_rules, fpgrowth_time)
)

ar.display_comparison(comparison)

COMPARAISON DES ALGORITHMES
             Métrique Apriori FP-Growth
  Ensembles fréquents      48        48
      Règles générées      17        17
Temps d'exécution (s)  0.0053    0.0086
     Vitesse relative   1.00x     0.62x



### Analyse comparative

#### Similarités

Les deux algorithmes produisent des résultats identiques :
- 48 ensembles fréquents
- 17 règles qui sont identiques
- Qualité identique : lift moyen de 6,205, confiance moyenne de 79,7%, support moyen de 6,2%

**Conclusion** : Les deux algorithmes implémentent la même logique de découverte de patterns fréquents ; par conséquent, avec des seuils identiques, ils produisent nécessairement les mêmes résultats.

#### Différences

Les algorithmes diffèrent au niveau des performances :
- La vitesse de FP-Growth varie considérablement. Sur 10 exécutions, j'ai observé qu'il était plus rapide qu'Apriori une seule fois.
- En moyenne, Apriori est 30-50% plus rapide que FP-Growth

Il s'agit d'un comportement inattendu, car FP-Growth est généralement considéré comme plus efficace parce qu'il ne génère pas de candidats. Cependant, compte tenu de la taille du dataset, ce résultat n'est pas surprenant : **FP-Growth performe mieux sur les grands datasets.**

Avec seulement 40 transactions, le coût de construction du FP-Tree dépasse les avantages de ne pas générer de candidats. L'approche plus simple d'Apriori est plus efficace pour des datasets très petits.

### Avantages de FP-Growth

- **Pas de génération de candidats** : Extrait les patterns fréquents directement de la structure arborescente
- **Structure compacte** : Le FP-Tree représente les transactions de manière condensée, évitant les lectures multiples de la base de données
- **Scalabilité** : Fonctionne extrêmement bien sur les grands datasets (>1000 transactions), car il construit l'arbre de décision une fois puis le parcourt simplement

## **7. Interprétation et recommandations**
### Génération des recommandations marketing basées sur les 5 principales règles

In [22]:
# Generate marketing recommendations
recommendations = ar.generate_recommendations(apriori_rules, min_lift=1.2, top_n=5)
ar.display_recommendations(recommendations)

RECOMMANDATIONS MARKETING
📌 Recommandation 1:
   Règle : {flour} --> {eggs}
   Lift : 10.000 | Confiance : 1.000 | Support : 0.050
   💡 Proposer une promotion groupée : {flour} + {eggs}

📌 Recommandation 2:
   Règle : {cereal} --> {banana, milk}
   Lift : 8.889 | Confiance : 0.667 | Support : 0.050
   💡 Proposer une promotion groupée : {cereal} + {banana, milk}

📌 Recommandation 3:
   Règle : {banana, milk} --> {cereal}
   Lift : 8.889 | Confiance : 0.667 | Support : 0.050
   💡 Proposer une promotion groupée : {banana, milk} + {cereal}

📌 Recommandation 4:
   Règle : {pasta} --> {tomato_sauce}
   Lift : 8.000 | Confiance : 0.800 | Support : 0.100
   💡 Proposer une promotion groupée : {pasta} + {tomato_sauce}

📌 Recommandation 5:
   Règle : {bread, tomato} --> {cheese}
   Lift : 8.000 | Confiance : 1.000 | Support : 0.050
   💡 Proposer une promotion groupée : {bread, tomato} + {cheese}





### Stratégies marketing proposées

Sur la base des règles découvertes, je propose les stratégies suivantes :

#### 1. Promotions groupées

Je propose d'offrir des promotions sur les groupes suivants, lorsqu'ils sont achetés ensemble :

- **Pack "Pâtisserie maison"** : Farine + oeufs (basé sur la règle {flour} --> {eggs}, lift = 10,0)
- **Pack "Dîner italien"** : Pâtes + sauce tomate (basé sur la règle {tomato_sauce} --> {pasta}, lift = 8,0)
- **Pack "Petit-déjeuner"** : Céréales + lait (basé sur la règle {cereal} --> {milk}, lift = 4,44)

#### 2. Placement en rayon

- Placer les pâtes sur la même ligne que la sauce tomate (forte association bidirectionnelle)
- Placer les céréales à proximité du lait, créant une zone petit-déjeuner
- Créer une zone goûter, avec le thé et le miel à proximité

#### 3. Recommandations personnalisées

- Lorsqu'un client place un article dans son panier, suggérer des articles avec lesquels il a une corrélation selon les règles, par exemple :
    - achat de farine --> suggérer des oeufs
    - achat de sauce tomate --> suggérer différents types de pâtes
    - achat de céréales --> suggérer du lait et des bananes
    - achat de pain --> suggérer du fromage et des tomates
- Inclure les paires mentionnées ci-dessus dans les programmes de fidélité en fonction de leurs habitudes d'achat

#### 4. Règles intéressantes

- Il y a 75% de chances qu'un client achète du lait lorsqu'il achète des bananes (règle : {banana} --> {milk}, lift = 3,33). Ceci n'est pas vraiment surprenant mais plutôt remarquable, car ce n'est pas une paire évidente.

**Conclusion** : La majorité des règles sont cohérentes, ce qui valide la qualité du dataset et confirme que les clients ont un plan de repas en tête lorsqu'ils font leurs courses. La paire banane + lait représente la minorité qui pourrait être intéressée par des repas plus spécifiques, tels que des smoothies (et non de simples spaghettis bolognaise ou da la tartine à la confiture).